In [34]:
import numpy as np
import pandas as pd
import matplotlib

import speech_recognition as sr
import soundfile
import librosa
import glob
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import pickle
import re
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

In [29]:
file = Path.cwd() / 'nlp-hesitation/fillers.txt'
with open(file, 'r') as text:
    data = text.read()
    data = re.sub(r"(\/\*.*?\*\/)", "", data, flags=re.M|re.DOTALL)
#print(data.strip())
fillers = data.split("\n")

In [43]:
audio = Path.cwd() / 'nlp-hesitation/ted-audiofile.wav'
r = sr.Recognizer()
#chunks the current audio file, sr to translate audio chunk to text
def transcribe(path):
    sound = AudioSegment.from_wav(path)  
    chunk_list = split_on_silence(sound, min_silence_len = 500, silence_thresh = sound.dBFS-14, keep_silence=500,)
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunk_list, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                pass
            else:
                text = f"{text.capitalize()}. "
                #print(chunk_filename, ":", text)
                whole_text += text
    return whole_text

In [ ]:
print("\nFull text:", transcribe(str(audio)))